In [9]:
# Monitor H transfer among NH2,NH molecules
# using coordination number
# returns a .xyz file
# written by Francesco Mambretti, 02/03/2023
# 16/03/2023 version

In [10]:
import sys
import os
import numpy as np
import ase
from ase.io import read, iread
from ase.io import write
import time as t

In [11]:
################################################################################################################
def load_traj (filename):
    traj = read(filename,index=':', parallel=True)
    
    return traj
    
##############################
def info_from_conf(conf,sym1,sym2):
    Ntot=len(conf)
    symbols=conf.get_chemical_symbols()
    N_c=sum(map(lambda x : x==sym1, symbols)) #centers
    N_n=sum(map(lambda x : x==sym2, symbols)) #neighbors

    return Ntot,N_c,N_n
    
##############################
def split_cen_neigh(atoms,sym1,sym2):

    atoms_c=ase.Atoms() #create two separate atoms objects
    atoms_n=ase.Atoms()
    for a in atoms:
        if a.symbol==sym1:
            atoms_c.append(a)
        elif a.symbol==sym2:
            atoms_n.append(a)
    atoms_c.set_cell(atoms.get_cell())
    atoms_c.pbc=True
    atoms_n.set_cell(atoms.get_cell())
    atoms_n.pbc=True

    #get indexes
    idx_c=[atom.index for atom in atoms if atom.symbol == sym1]
    idx_n=[atom.index for atom in atoms if atom.symbol == sym2]
    
    return atoms_c,atoms_n,idx_c,idx_n
    
##############################


In [118]:
def check_rebuild(atoms,idx_c,idx_prev_n,cutoff_build): #decide whether to recompute center-neighbor distances or not

    #loop over center atoms
    ii=0
    for i in idx_c: #compute such distances only for center atoms
        for j in idx_prev_n[ii]:
            r=atoms.get_distance(i,j,mic=True)
            #if (i==108 or i==101):
            #    print(r,j)
            if (r > cutoff_build):
                #print(r)
                return True
            
        ii+=1
        
    return False
    
##############################

def compute_coord(N_c,N_n,atoms_c,atoms_n,idx_c,idx_n,atoms,box,cutoff,rij):

    #get all distances between the selected centers and the selected neighbors
    ii=0
    
    for i in idx_c: #compute such distances only for center atoms
        rij[ii]=(atoms.get_distances(i,idx_n,mic=True)) #get all the distances from neighbors
        ii+=1 #next row
        
    coord_array=np.empty(0)
    
    idx_prev_n=[]
    
    for i in range(N_c): #loop over rows
        idx_ok = np.argwhere( (rij[i] < cutoff) ) [:,0]
        coord_array=np.append(coord_array,len(idx_ok))
        
        list_ok=[]
        for id in idx_ok:
            list_ok.append(idx_n[id])
        
            #print(list_ok)
        idx_prev_n.append(list_ok)

    return coord_array, idx_prev_n


In [127]:
##############################

def run_analysis(folder,traj,elem_c,elem_n,cutoff,cutoff_build,outfile):
    
    step=0
    count_pt=0
    
    for atoms in traj[:]: #do the full analysis (without plots)
       # print(atoms)
        mycell=atoms.cell
        box=mycell.lengths()
        
        if(step==0):
            Ntot,N_c,N_n=info_from_conf(atoms,elem_c,elem_n) #extract info from config
            rij=np.zeros((N_c,N_n)) #centers-neighbors distances
            coord_array=np.empty(N_c)
            atoms_c,atoms_n,idx_c,idx_n=split_cen_neigh(atoms,elem_c,elem_n) # split into centers and neighbors
            #print(idx_c)
            atoms.set_pbc(True)
            atoms.set_cell(box)
        
        coord_prev=coord_array #save previous step
        
        if(step!=0):
            rebuild=check_rebuild(atoms,idx_c,idx_prev_n,cutoff_build)
        else:
            rebuild=True
            
        if (rebuild==True):
            print("rebuilding at step..."+str(step))
            coord_array,idx_prev_n=compute_coord(N_c,N_n,atoms_c,atoms_n,idx_c,idx_n,atoms,box,cutoff,rij) #compute NH coordination (NH-c)
        atoms_c.set_array('coordNH',coord_array)
                
        write(outfile,atoms_c,append=True) #re-write trajectory with NH-c
        
        if(step==6179):
            print(coord_array)
            
        #detect transfers
        if (np.array_equal(coord_array,coord_prev)==False):
            diff_array = np.where(coord_array!=coord_prev)
            if(step==6180):
                print(coord_prev)
    
            i=0
            print("Proton transfer at step "+str(step))  #", one H moved from N #{} to #{}".format(,))
            count_pt+=1
            
            if(step!=0):
                for i in (np.asarray(diff_array)).T:
                    print ("{:.3f} {:.3f} {:.3f}".format(idx_c[i[0]+1],coord_array[i[0]],coord_prev[i[0]])) #+1 is for ovito indexing
                    print(coord_array,coord_prev)
                
                #print(diff_array)
                    i=i+1
        
        step+=1

    print("In total, {} proton transfers have occurred".format(count_pt))
    return
################################################################################################################

In [53]:
#read traj

input_file="12_NH2/run_0/dump.lammpstrj"

folder="./"
time0=t.perf_counter()
traj=load_traj(folder+"/"+input_file)
time1=t.perf_counter()
    
print(str(time1-time0)+" s to read the trajectory")

101.67527778912336 s to read the trajectory


In [128]:
outfile="12_NH2/run_0/dump/dump.ext.xyz"
cutoff=1.5 #A
cutoff_build=1.3 #if one N-H bond gets longer than this, rebuild lists

os.system("test -f "+str(outfile)+" && rm "+str(outfile))

run_analysis(folder,traj,"He","H",cutoff,cutoff_build,outfile) #adjust without chem symb

rebuilding at step...0
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1.
 1. 1. 1. 1. 2. 1. 1. 1. 1. 1. 1. 1. 2. 1. 1. 1. 1. 1. 1. 1. 1. 1. 2. 2.
 1. 1. 1. 1. 1. 1. 2. 1. 1. 1. 1. 1. 1. 1. 1. 1. 2. 1. 1. 1. 1. 1. 1. 2.
 1. 2. 1. 1. 1. 2. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 2. 1. 1. 1. 1. 1. 2.]
rebuilding at step...6180
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1.
 1. 1. 1. 1. 2. 1. 1. 1. 1. 1. 1. 1. 2. 1. 1. 1. 1. 1. 1. 1. 1. 1. 2. 2.
 1. 1. 1. 1. 1. 1. 2. 1. 1. 1. 1. 1. 1. 1. 1. 1. 2. 1. 1. 1. 1. 1. 1. 2.
 1. 2. 1. 1. 1. 2. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 2. 1. 1. 1. 1. 1. 2.]
Proton transfer at step 6180
79.000 1.000 0.000
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 2. 1. 1. 1. 1. 1. 1. 2. 1. 1. 2. 2.
 1. 1. 1. 1. 1. 1. 2. 1. 1. 1. 1. 1. 1. 1. 1. 1. 2. 1. 1. 1. 1. 1. 1. 2.
 1. 2. 1. 1. 1. 2. 1. 1. 1. 1. 1. 1. 2. 1. 1. 1. 1. 2. 1. 1. 1. 1. 1. 2.] [1. 1. 1. 1. 1. 1. 1. 1.

KeyboardInterrupt: 